In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import glob
import os
import warnings

from astropy.io import ascii
from astropy.io import fits

%matplotlib inline

In [2]:
igrins_log = pd.read_csv('./reduced_log.csv')
igrins_log = igrins_log.drop(labels=['SNRH','SNRK','ROTPA','AM','BVC'],axis=1)

In [4]:
igrins_log

,NAME,OBJNAME_super,OBJNAME_recipe,RA.2000,DEC.2000,FILENAME,FILES,URLS_H,URLS_RTELL_H,URLS_K,...,CIVIL,FILENUMBER,JD,OBJTYPE,EXPTIME,FACILITY,PI,NIGHT_URLS,OTHER_URLS,PAPER_LOGS
0,* mu.02 Her,LHS 3325,LHS 3325,266.771375,27.727028,SDCH_20140707_0095.fits,95 96 97 98,{'20140707/SDCH_20140707_0095.sn.fits': 'https...,{'20140707/SDCH_20140707_0095.rtell_H.center_a...,{'20140707/SDCK_20140707_0095.sn.fits': 'https...,...,20140707,95,2.456847e+06,TAR,90.00,McDonald,Commissioning,{'20140707/a0v.db': 'https://utexas.box.com/sh...,{'20140707/SDCH_20140707_0095.slit_profile.jso...,['https://utexas.box.com/shared/static/q1k4g10...
1,* alf Lyr,vega,vega,279.393667,38.804667,SDCH_20140707_0099.fits,99 100 101 102 103 104 105 106 107 108 109 110,{'20140707/SDCH_20140707_0099.sn.fits': 'https...,NaN,{'20140707/SDCK_20140707_0099.sn.fits': 'https...,...,20140707,99,2.456847e+06,STD,1.63,McDonald,Commissioning,{'20140707/a0v.db': 'https://utexas.box.com/sh...,{'20140707/SDCH_20140707_0099.slit_profile.jso...,['https://utexas.box.com/shared/static/q1k4g10...
2,HD 164595B,NLTT45791,NLTT45791,270.353292,29.581639,SDCH_20140707_0111.fits,111 112 113 114,{'20140707/SDCH_20140707_0111.sn.fits': 'https...,{'20140707/SDCH_20140707_0111.rtell_H.center_a...,{'20140707/SDCK_20140707_0111.sn.fits': 'https...,...,20140707,111,2.456847e+06,TAR,240.00,McDonald,Commissioning,{'20140707/a0v.db': 'https://utexas.box.com/sh...,{'20140707/SDCH_20140707_0111.slit_profile.jso...,['https://utexas.box.com/shared/static/q1k4g10...
3,BD+31 3330B,NLTT46858,NLTT46858,280.389208,31.558806,SDCH_20140707_0116.fits,116 117 118 119,{'20140707/SDCH_20140707_0116.sn.fits': 'https...,{'20140707/SDCH_20140707_0116.rtell_H.center_a...,{'20140707/SDCK_20140707_0116.sn.fits': 'https...,...,20140707,116,2.456847e+06,TAR,90.00,McDonald,Commissioning,{'20140707/a0v.db': 'https://utexas.box.com/sh...,{'20140707/SDCH_20140707_0116.slit_profile.jso...,['https://utexas.box.com/shared/static/q1k4g10...
4,L 1288-4,GJ797B,GJ797B,310.365000,19.958444,SDCH_20140707_0124.fits,124 125 126 127 128 129 130 131,{'20140707/SDCH_20140707_0124.sn.fits': 'https...,{'20140707/SDCH_20140707_0124.rtell_H.center_a...,{'20140707/SDCK_20140707_0124.sn.fits': 'https...,...,20140707,124,2.456847e+06,TAR,120.00,McDonald,Commissioning,{'20140707/a0v.db': 'https://utexas.box.com/sh...,{'20140707/SDCH_20140707_0124.slit_profile.jso...,['https://utexas.box.com/shared/static/q1k4g10...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13567,* 60 Ori,HIP 28296,HIP 28296,89.706133,0.552617,SDCH_20211230_0033.fits,33 34 35 36,{'20211230/SDCH_20211230_0033.sn.fits': 'https...,{'20211230/SDCH_20211230_0033.rtell_H.spec_a0v...,{'20211230/SDCK_20211230_0033.sn.fits': 'https...,...,20211230,33,2.459580e+06,TAR,4.00,Gemini South,Gemini,{'20211230/a0v.db': 'https://utexas.box.com/sh...,{'20211230/SDCH_20211230_0033.slit_profile.jso...,NaN
13568,ATO J014.1326-08.1515,UCAC2 28842571,UCAC2 28842571,14.132262,-8.152219,SDCH_20211231_0021.fits,21 22 23 24,{'20211231/SDCH_20211231_0021.sn.fits': 'https...,{'20211231/SDCH_20211231_0021.rtell_H.center_a...,{'20211231/SDCK_20211231_0021.sn.fits': 'https...,...,20211231,21,2.459581e+06,TAR,220.00,Gemini South,Gemini,{'20211231/a0v.db': 'https://utexas.box.com/sh...,{'20211231/SDCH_20211231_0021.slit_profile.jso...,NaN
13569,* 28 Cet,HIP 5164,HIP 5164,16.521158,-9.839789,SDCH_20211231_0029.fits,29 30 31 32,{'20211231/SDCH_20211231_0029.sn.fits': 'https...,{'20211231/SDCH_20211231_0029.rtell_H.spec_a0v...,{'20211231/SDCK_20211231_0029.sn.fits': 'https...,...,20211231,29,2.459581e+06,TAR,6.00,Gemini South,Gemini,{'20211231/a0v.db': 'https://utexas.box.com/sh...,{'20211231/SDCH_20211231_0029.slit_profile.jso...,NaN
13570,NaN,TIC 434260570,TIC 434260570,64.243296,14.771306,SDCH_20211231_0034.fits,34 35 36 37,{'20211231/SDCH_20211231_0034.sn.fits': 'https...,{'20211231/SDCH_20211231_0034.rtell_H.center_a...,{'20211231/SDCK_20211231_0034.sn.fits': 'https

In [5]:
pd.read_csv('./apjs328021t1_coords_ascii.txt')

,Table 1
0,IRAC Observing Log
1,AOR\tPID\tDate\t3.6/5.8 Center\t4.5/8.0 Center...
2,\t\t(UT)\t(J2000)\t(J2000)\t(arcmin)\t(deg)\t(...
3,3963904\t37\t2004 Feb 9\t04 46 42.3 +24 58 58\...
4,\t\t\t04 46 46.6 +24 52 21\t04 46 38.5 +25 05 ...
...,...
524,26475520\t50584\t2008 Nov 2\t04 40 53.4 +20 55...
525,\t\t\t04 40 51.6 +21 02 39\t04 40 57.0 +20 49 ...
526,Notes.
527,a Position angle of the long axis of the map.
